In [1]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 57 bits virtual
  Byte Order:                Little Endian
CPU(s):                      64
  On-line CPU(s) list:       0-63
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) Platinum 8358 CPU @ 2.60GHz
    CPU family:              6
    Model:                   106
    Thread(s) per core:      1
    Core(s) per socket:      32
    Socket(s):               2
    Stepping:                6
    CPU(s) scaling MHz:      24%
    CPU max MHz:             3400.0000
    CPU min MHz:             800.0000
    BogoMIPS:                5200.00
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush dts acpi mmx fxsr sse 
                             sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm cons
                             tant_tsc art arch_perfmo

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pickle
import time
import seaborn as sns
from scipy.optimize import fmin, minimize, LinearConstraint, Bounds

from efficient_fpt.multi_stage_cy import compute_loss_parallel, print_num_threads

# data generated with addm_gen_data.ipynb in example4
data = pickle.load(open("addm_data_20251012-175957.pkl", "rb"))

In [3]:
CYTHON_TYPE = np.float64

a = data["a"]
b = data["b"]
x0 = data["x0"]
mu1_true = data["mu1"]
mu2_true = data["mu2"]
sigma = data["sigma"]
T = data["T"]

mu_true_data = data["mu_data_padded"].astype(CYTHON_TYPE)
sacc_data = data["sacc_data_padded"].astype(CYTHON_TYPE)
length_data = data["d_data"].astype(np.int32)
rt_data = data["decision_data"][:, 0].astype(CYTHON_TYPE)
choice_data = data["decision_data"][:, 1].astype(np.int32)

num_data, max_d = mu_true_data.shape
# set data range if necessary
start_index, end_index = 0, 50000
num_data = end_index - start_index
mu_true_data, sacc_data, length_data, rt_data, choice_data = mu_true_data[start_index:end_index], sacc_data[start_index:end_index], length_data[start_index:end_index], rt_data[start_index:end_index], choice_data[start_index:end_index]
flag_data = np.isclose(mu_true_data[:, 0], mu2_true).astype(np.int32)


In [4]:
a, b, x0, max(rt_data), np.mean(length_data)

(1.5, 0.0, -0.2, 18.72120499946043, 3.80632)

In [5]:
mu1_true_data = np.full(num_data, mu1_true, dtype=CYTHON_TYPE)
mu2_true_data = np.full(num_data, mu2_true, dtype=CYTHON_TYPE)

In [6]:
num_iter = 10
start_time = time.time()
for _ in range(num_iter):
    loss = compute_loss_parallel(mu1_true_data, mu2_true_data, rt_data, choice_data, flag_data, sacc_data, length_data, max_d, sigma, a, b, x0)
end_time = time.time()
print(f"Likelihood evaluation time: {(end_time - start_time) / num_iter:.3f} s")

Likelihood evaluation time: 0.838 s


In [7]:
print("\n")
print_num_threads()
print("# data =", num_data)

# Constraint optimization for searching all parameters
print("\nNumerical optimization for mu1, mu2:")
method = "Nelder-Mead"
print("Using " + method)
func = lambda paras: compute_loss_parallel(np.full(num_data, paras[0], dtype=CYTHON_TYPE), 
                                           np.full(num_data, paras[1], dtype=CYTHON_TYPE), 
                                           rt_data, choice_data, flag_data, sacc_data, length_data, max_d, sigma, a, b, x0)
initial_guess = [0, 0]
print("Initial guess:", initial_guess)
print()
start_time = time.time()
paras_opt_result = minimize(func, x0=initial_guess, method=method)
print(f"Total time: {time.time() - start_time:.3f} seconds")
print(paras_opt_result)



Number of available threads: 64
# data = 50000

Numerical optimization for mu1, mu2:
Using Nelder-Mead
Initial guess: [0, 0]

Total time: 100.276 seconds
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 2.0141445849026733
             x: [ 1.002e+00 -7.960e-01]
           nit: 61
          nfev: 121
 final_simplex: (array([[ 1.002e+00, -7.960e-01],
                       [ 1.002e+00, -7.959e-01],
                       [ 1.002e+00, -7.959e-01]]), array([ 2.014e+00,  2.014e+00,  2.014e+00]))


In [8]:
theta_hat = paras_opt_result['x']
print("True and estimated value of parameters:")
print(f"mu1: {mu1_true:.4f}, {theta_hat[0]:.4f}")
print(f"mu2: {mu2_true:.4f}, {theta_hat[1]:.4f}")


True and estimated value of parameters:
mu1: 1.0000, 1.0025
mu2: -0.8000, -0.7960


#### TADA "maximum likelihood" estimator

In [9]:
from efficient_fpt.multi_stage_cy import compute_tadaloss_parallel

In [10]:
print("\n")
print_num_threads()
print("# data =", num_data)

# Constraint optimization for searching all parameters
print("\nNumerical optimization for mu1, mu2:")
method = "Nelder-Mead"
print("Using " + method)
func = lambda paras: compute_tadaloss_parallel(np.full(num_data, paras[0], dtype=CYTHON_TYPE), 
                                           np.full(num_data, paras[1], dtype=CYTHON_TYPE), 
                                           rt_data, choice_data, flag_data, sacc_data, length_data, max_d, sigma, a, b, x0)
initial_guess = [0, 0]
print("Initial guess:", initial_guess)
print()
start_time = time.time()
paras_opt_result = minimize(func, x0=initial_guess, method=method)
print(f"Total time: {time.time() - start_time:.3f} seconds")
print(paras_opt_result)



Number of available threads: 64
# data = 50000

Numerical optimization for mu1, mu2:
Using Nelder-Mead
Initial guess: [0, 0]

Total time: 105.884 seconds
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1.9776291427908943
             x: [ 1.684e+00 -1.478e+00]
           nit: 64
          nfev: 123
 final_simplex: (array([[ 1.684e+00, -1.478e+00],
                       [ 1.684e+00, -1.478e+00],
                       [ 1.684e+00, -1.478e+00]]), array([ 1.978e+00,  1.978e+00,  1.978e+00]))


In [11]:
theta_hat = paras_opt_result['x']
print("True and estimated value of parameters:")
print(f"mu1: {mu1_true:.3f}, {theta_hat[0]:.4f}")
print(f"mu2: {mu2_true:.3f}, {theta_hat[1]:.4f}")

True and estimated value of parameters:
mu1: 1.000, 1.6838
mu2: -0.800, -1.4782
